In [ ]:
from config import *
import pandas as pd
import requests
import json
import urllib
import dateutil.parser
import time
from datetime import datetime

In [ ]:
# underlying stock ticker symbol
ticker = 'SPY'

# expiration date of desired option where last number '7' is the days until expiration
exp_date = '2023-01-20:7'

# strike price of desired option in str
strike_price = '395.0'

# sets the desired net delta. net delta = number of share + option delta
target_net_delta = -2.0


# sets time interval (in seconds) for bot to check position and make adjustments in seconds
time_interval = 10

In [ ]:
# Define TDAmeritrade Endpoint and Payload
tda_endpoint = 'https://api.tdameritrade.com/v1/marketdata/chains'
tda_payload = {'apikey': TDA_ID,
               'symbol': ticker,
               'contractType': 'CALL',
               'includeQuotes': 'TRUE',
               'strategy': 'SINGLE',
               'expMonth': 'ALL',
               "isDelayed": 'true'}

# define alpaca endpoint and url
HEADERS = {'APCA-API-KEY-ID': ALPACA_KEY_ID, 'APCA-API-SECRET-KEY': ALPACA_SECRET_KEY}
alpaca_endpoint = 'https://paper-api.alpaca.markets'
account_url = '{}/v2/account'.format(alpaca_endpoint)
orders_url = '{}/v2/orders'.format(alpaca_endpoint)
positions_url = '{}/v2/positions'.format(alpaca_endpoint)
portfolio_url = '{}/v2/account/portfolio/history'.format(alpaca_endpoint)

In [1]:
# this function will get the profit/loss on the stock position in alpaca
def get_pnl():
    data = {
        'period': '1D',
        'timeframe': '1D'
    }
    r = requests.get(portfolio_url, headers=HEADERS)
    return json.loads(r.content)


# returns the number of shares currently open
def num_of_shares(ticker):
    r = requests.get(positions_url + '/{}'.format(ticker), headers=HEADERS)
    return json.loads(r.content)


# puts in an order to buy/sell shares. side='buy' or 'sell'
def create_order(ticker, qty, side): 
    data = {
        'symbol': ticker,
        'qty': qty,
        'side': side,
        'type': 'market',
        'time_in_force': 'day'
    }
    r = requests.post(orders_url, json=data, headers=HEADERS)
    

# returns the delta of the short call
def get_call_delta(tda_data, exp_date, strike_price):
    return tda_data['callExpDateMap'][exp_date][strike_price][0]['delta'] * -100


# closes all alpaca positions
def close_all_positions():
    r = requests.delete(positions_url, headers=HEADERS)

In [ ]:
# this returns how much you sold the option for in nominal dollars
r = requests.get(url=tda_endpoint, params=tda_payload)
tda_data = json.loads(r.content)
option_sell_price = tda_data['callExpDateMap'][exp_date][strike_price][0]['bid'] * 100

In [ ]:
while (datetime.now().hour * 60 + datetime.now().minute) < 897: # bot keeps running until 3 minutes before market close, local time
    # gets the number of shares currently open 
    try:
        stock_delta = float(num_of_shares(ticker)['qty'])
    except:
        stock_delta = 0
    
    # gets the option delta and net_delta
    try:
        short_call_delta = get_call_delta(tda_data, exp_date, strike_price)
        current_net_delta = round(stock_delta + short_call_delta, 0)
        if current_net_delta < target_net_delta:
            create_order(ticker, (target_net_delta - current_net_delta), 'buy')
        elif current_net_delta > target_net_delta:
            create_order(ticker, (current_net_delta - target_net_delta), 'sell')
    except:
        print('error')
    
    print('open shares: ', stock_delta)
    print('option delta: ', short_call_delta)
    print('net delta: ', current_net_delta)
    print('option profit/loss: ', option_sell_price - (tda_data['callExpDateMap'][exp_date][strike_price][0]['ask'] * 100))
    print('stock profit/loss: ', get_pnl()['profit_loss'][-1])
    print(datetime.now(), '\n')
    print()
    
    time.sleep(time_interval) 

# after market closes, all alpaca positions will be closed
close_all_positions()